#Modules Importation and preparation

In [ ]:
WORK_DIR = "."
% cd "."

In [ ]:
import Clusterer, Global_feature_exractors, Local_features_extractor, Norms, Image, PCA_reduction, Distances
from Dataset_loader import train_test_IAM, train_test_ICDAR2013, train_test_TrigraphSlant
from Accuracy import accuracy_optimised
import json, os
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
training_session = "training_16Sep"

with open("./config_train.json") as config_file:
    configuration = json.load(config_file)["Sift_Bow"][training_session]

CLUSTERS_CENTERS_PATH = training_session+"/centers_clusters_BOW.npy"

if not os.path.exists(training_session):
  os.mkdir(training_session)

In [ ]:
train, test = train_test_TrigraphSlant("./dataset/TrigraphSlant", size_train=configuration["train_size"], size_test=configuration["test_size"])
_, _, images_train_set = map(list, zip(*train))
writers_test_set, images_names_test_set, images_test_set = map(list, zip(*test))

#Train Model

In [ ]:
def choose_number_clusters(images, max_no_improvement, test_values=range(1, 500, 50)):
  hellinger_normalization = Norms.Norm.hellinger_normalization
  sift = Local_features_extractor.Local_feature_exractor(hellinger_normalization)
  images_pre = [Image.Image(image, local_feature_extractor=sift, verbose=False) for image in images]

  Clusterer.Clusterer.elbow_method_kmeans(images_pre, 
                                          max_no_improvement=max_no_improvement, 
                                          test_values=test_values)

In [ ]:
def save_new_clusters_centers(images, path_to_save, nb_clusters, max_no_improvement):
	hellinger_normalization = Norms.Norm.hellinger_normalization
	sift = Local_features_extractor.Local_feature_exractor(hellinger_normalization)
	images_pre = [Image.Image(image, local_feature_extractor=sift, verbose=False) for image in images]
 
	Clusterer.Clusterer.fit_new_trainig(images_pre, 
																			path_to_save, 
																			nb_clusters=nb_clusters, 
																			max_no_improvement=max_no_improvement,
																			verbose=0)

In [ ]:
choose_number_clusters(images=images_train_set, 
                       max_no_improvement=configuration["max_no_improvement"],
                       test_values = range(1, 300, 50))

In [ ]:
nb_clusters_kmeans = 50

In [ ]:
save_new_clusters_centers(images=images_train_set,
                          path_to_save=CLUSTERS_CENTERS_PATH, 
                          nb_clusters=nb_clusters_kmeans,
                          max_no_improvement=configuration["max_no_improvement"])

#Test Model

In [ ]:
import importlib, Dataset_loader
importlib.reload(Clusterer)

In [ ]:
def test_sift_descr_bow(images, images_names, writers, cluster_centers_path, accuracy_calculator):
  
  hellinger_normalization = Norms.Norm.hellinger_normalization
  sift = Local_features_extractor.Local_feature_exractor(hellinger_normalization)

  clusters_centers = Clusterer.Clusterer.fit_ancient_data(cluster_centers_path)
  bow = Global_feature_exractors.BOW(clusters_centers)

  images_pre = [Image.Image(image, image_name=image_name, local_feature_extractor=sift, global_feature_extractor=bow) for image, image_name in zip(images,images_names)]

  chi2_distance = Distances.Distance.chi2_distance
  accuracy_value = accuracy_calculator(X_test=images_pre, 
                                       Y_test=writers, 
                                       global_feature_extractor=bow, 
                                       distance_metric=chi2_distance)
  return accuracy_value

In [ ]:
accuracy_value= test_sift_descr_bow(images=images_test_set, 
                                    images_names=images_names_test_set,
                                    writers=writers_test_set, 
                                    cluster_centers_path=CLUSTERS_CENTERS_PATH,
                                    accuracy_calculator=accuracy_optimised)

print()
print("Accuracy value for <", training_session,"> : ","{:.2%}".format(accuracy_value), sep="")